# Assignment 2
**COMP 511: Network Science**

**Due on February 6th 2026**

In [ ]:
import pandas as pd
import networkx as nx
from collections import Counter, defaultdict
import gdown
import tarfile
import os
import re
import numpy as np

## 1. Centrality Measures [20%]

Select 3 (or more) centrality measures and find the top 5 most important nodes in the [Enron dataset](https://www.cs.cornell.edu/~arb/data/pvc-email-Enron/). Who are the top ranked people?

*   aggregate all emails sent at different times into a static snapshot with an edge weight showing how many emails in total have been send from one node to the other

In [ ]:
# google drive file ID
file_id = "1hbpdnqvTRqPZ4MvzxeKLLkifdXZhtQsn"

output_file = "email-Enron.tar.gz"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file, quiet=False)

# Extract the .tar.gz file
try:
    with tarfile.open(output_file, "r:gz") as tar:
        tar.extractall()  # extract into "email-Enron" folder
        print("Extraction completed successfully.")
except tarfile.ReadError as e:
    print("Error reading the tar.gz file:", e)

In [ ]:
data_folder = 'email-Enron'
addresses_path = f"{data_folder}/addresses-email-Enron.txt"
core_path = f"{data_folder}/core-email-Enron.txt"
email_path = f"{data_folder}/email-Enron.txt"

# read the addresses
addresses_df = pd.read_csv(
    addresses_path,
    sep=r'\s+',
    header=None,
    names=['ID', 'email_address'],
    on_bad_lines='skip'
)
addresses_df.set_index('ID', inplace=True)

# read core ids
core_df = pd.read_csv(
    core_path,
    sep=r'\s+',
    header=None,
    names=['ID']
)

# use set for efficiency
core_ids = set(core_df['ID'].tolist())

# read email edges
edges_df = pd.read_csv(
    email_path,
    sep=r'\s+',
    header=None,
    names=['sender', 'recipient', 'timestamp']
)

# we need to aggregate the edge weights to get a static snapshot that we can use
agg_edges = edges_df.groupby(['sender', 'recipient']).size().reset_index(name='weight')

# now lets make the static graph
G_static = nx.DiGraph()

# add weights between nodes based on agg_edges
for row in agg_edges.itertuples(index=False):
    G_static.add_edge(row.sender, row.recipient, weight=row.weight)

# add the email as a node attribute so we can see important names later
for node in G_static.nodes():
  try:
    G_static.nodes[node]['email_address'] = addresses_df.loc[node, 'email_address']
  except KeyError:
    pass

In [ ]:
# Write your code here to calculate centrality measures and find top 5 nodes


## 2. Graph Clustering [50%]

Select two (or more) community detection or graph clustering algorithms, apply them on the following real world datasets and evaluate their performances:

*   -- real-classic: [strike, karate, polblog, polbooks, football](http://www.reirab.com/Teaching/NS20/classic.tar.gz)
*   -- real-node-label: [citeseer, cora, pubmed](https://github.com/tkipf/gcn/tree/master/gcn/data)

Here, the goal is to use the classification labels as clustering labels, and see how well we can find those labels without using the feature vectors.

In [ ]:
# download classic graph data
download_folder = "/content/classic_graph_data/"
os.makedirs(download_folder, exist_ok=True)

file_ids = {
    "polbooks.gml": "1SzhpzmpEO9NSjEabjoZP16NzYN3s6RBn",
    "polbooks.txt": "1yEzPMsGPUwrGepoxN4hlX3a54CvAPLj-",
    "polblogs.gml": "10ptbBiMgm541aAr3Rd4svBFia6qjJOGV",
    "polblogs.txt": "1un9r36q4qtuz9Kjv7MqVifMp0pH45Es2",
    "karate.gml": "1zvpOcA_74qO0pp9o36xg2rdwx9pTkgW_",
    "karate.txt": "1-i66II9DThEJruAWoslgb2Cc88R17jBK",
    "strike.gml": "137lB6IHqkRzHYOeYBn-0n4pNIYN3tuvr",
    "football.gml": "1ooEj5eBI7UM2ZtfexmaTxIFJt6qn43Uy",
    "football.txt": "126uHBR218fWCBRJjORSmaaxoHUGfTbPp",
}

for file_name, file_id in file_ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    output_path = os.path.join(download_folder, file_name)
    gdown.download(url, output_path, quiet=False)
    print(f"Downloaded: {file_name}")

# download labeled graph data
download_folder = "/content/labeled_graph_data/"
os.makedirs(download_folder, exist_ok=True)

file_ids = {
    "citeseer_node_labels": "1u51oSH8O46i3WnG_RqHrxcZnTURod20l",
    "citeseer_edges": "1IceUHrm8fk51Fb_gLHmXg0kETcCZa2bw",
    "cora_cites": "1AC_LOmycApD7_MtahW6WeaxSOzhPPp9J",
    "cora_content": "1mQ9dWt_8jJr0pJFiyTsKS2hGEE85YPZQ",
    "pubmed_node_labels": "1HUlVeZCjiO4GEQrQgH282VPC835mfPaO",
    "pubmed_edges": "1lss1_snvRj8sx4Aoo-vS2bJt62j_3YPt"
}

for file_name, file_id in file_ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    output_path = os.path.join(download_folder, file_name)
    gdown.download(url, output_path, quiet=False)
    print(f"Downloaded: {file_name}")

In [ ]:
# nx.read_gml needs a simple graph so we have to manually remove duplicate edges
def clean_gml_file(input_path, output_path, gml_filename):

    with open(input_path, 'r') as f:
        lines = f.readlines()

    seen_edges = set()
    new_lines = []
    inside_edge_block = False

    # edge block
    buffer_block = []
    edge_counter = 0

    for line in lines:
        stripped_line = line.rstrip("\n\r")

        # detect start of an edge block
        if stripped_line.strip().startswith("edge ["):
            inside_edge_block = True
            buffer_block = [line]  # start buffering this edge block
            continue

        if inside_edge_block:
            buffer_block.append(line)

            if stripped_line.strip().startswith("]"):
                # end of the edge block
                inside_edge_block = False
                edge_counter += 1

                # combine block into one string for parsing
                block_text = "".join(buffer_block)

                #extract source & target from any line within the block
                match_source = re.search(r'\bsource\s+(\d+)', block_text)
                match_target = re.search(r'\btarget\s+(\d+)', block_text)

                if match_source and match_target:
                    source = int(match_source.group(1))
                    target = int(match_target.group(1))

                    if source != target:  # self-loops
                        edge_tuple = tuple(sorted((source, target)))

                        if edge_tuple not in seen_edges:
                            seen_edges.add(edge_tuple)
                            new_lines.extend(buffer_block)

                # reset buffer
                buffer_block = []

        else:
            # non-edge lines go directly to output
            new_lines.append(line)

    # save the cleaned gml file
    with open(output_path, 'w') as out:
        out.writelines(new_lines)

    print(f"[CLEAN] Removed duplicates. Cleaned file saved to: {output_path}")

classic_graph_folder = "/content/classic_graph_data/"
classic_cleaned_folder = "/content/classic_graph_data_cleaned/"
os.makedirs(classic_cleaned_folder, exist_ok=True)

classic_gml_files = [f for f in os.listdir(classic_graph_folder) if f.endswith(".gml")]
classic_graphs = {}

print("Loading 'classic' graphs. Note: These are often referred to as unlabeled, but most (except strike) do have ground truth labels available.")

for gml_file in classic_gml_files:

    #clean all the gml files first
    input_path = os.path.join(classic_graph_folder, gml_file)
    output_path = os.path.join(classic_cleaned_folder, gml_file)
    clean_gml_file(input_path, output_path, gml_file)

    # load and apply some processing
    try:
        G = nx.read_gml(output_path)

        # undirected simple graph
        if isinstance(G, nx.MultiDiGraph) or isinstance(G, nx.DiGraph):
            G = nx.Graph(G)

        # remove self-loops (double checking)
        G.remove_edges_from(nx.selfloop_edges(G))

        # Rename 'value' to 'label' if present
        for node, data in G.nodes(data=True):
            if 'value' in data:
                G.nodes[node]['label'] = data['value']

        classic_graphs[gml_file] = G
        print(f"[LOADED] {gml_file} -> {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")
    except nx.NetworkXError as e:
        print(f"[ERROR] Could not load {gml_file}: {e}")

In [ ]:
# now we load the labeled graphs
labeled_graph_folder = "/content/labeled_graph_data/"
labeled_cleaned_folder = "/content/labeled_graph_data_cleaned/"
os.makedirs(labeled_cleaned_folder, exist_ok=True)

graph_folder = "labeled_graph_data"
edges_file = os.path.join(graph_folder, "citeseer_edges")
labels_file = os.path.join(graph_folder, "citeseer_node_labels")

def load_citeseer_graph(edges_file, labels_file):
    G = nx.Graph()

    with open(edges_file, "r") as f:
        for line in f:
            src, dst, *_ = line.strip().split(",")
            G.add_edge(int(src), int(dst))

    labels = {}
    with open(labels_file, "r") as f:
        for line in f:
            node, label = line.strip().split(",")
            node, label = int(node), int(label)
            G.nodes[node]["label"] = label
            labels[node] = label

    return G, labels

G_citeseer, citeseer_labels = load_citeseer_graph(edges_file, labels_file)
print(f"Loaded CiteSeer graph with {G_citeseer.number_of_nodes()} nodes and {G_citeseer.number_of_edges()} edges.")

# Now we do the cora graph
edges_file = os.path.join(graph_folder, "cora_cites")
content_file = os.path.join(graph_folder, "cora_content")

def load_cora_graph(edges_file, content_file):
    G = nx.Graph()
    node_features = {}
    node_labels = {}

    with open(content_file, "r") as f:
        for line in f:
            parts = line.strip().split("\t")  # tab separator
            node_id = int(parts[0])  # first column is node ID
            features = np.array([int(x) for x in parts[1:-1]])  # middle columns are features
            label = parts[-1]  # last column is class label

            G.add_node(node_id, features=features, label=label)
            node_features[node_id] = features
            node_labels[node_id] = label

    with open(edges_file, "r") as f:
        for line in f:
            src, dst = map(int, line.strip().split("\t"))
            G.add_edge(src, dst)

    return G, node_features, node_labels

G_cora, cora_features, cora_labels = load_cora_graph(edges_file, content_file)
print(f"Loaded Cora graph with {G_cora.number_of_nodes()} nodes and {G_cora.number_of_edges()} edges.")

# Now pubmed
edges_file = os.path.join(graph_folder, "pubmed_edges")
labels_file = os.path.join(graph_folder, "pubmed_node_labels")

def load_pubmed_graph(edges_file, labels_file):
    G = nx.Graph()

    with open(edges_file, "r") as f:
        for line in f:
            src, dst = map(int, line.strip().split(","))
            G.add_edge(src, dst)

    labels = {}
    with open(labels_file, "r") as f:
        for line in f:
            node, label = line.strip().split(",")
            node, label = int(node), int(label)
            G.nodes[node]["label"] = label
            labels[node] = label

    return G, labels

G_pubmed, pubmed_labels = load_pubmed_graph(edges_file, labels_file)
print(f"Loaded PubMed graph with {G_pubmed.number_of_nodes()} nodes and {G_pubmed.number_of_edges()} edges.")

### 2(a) Algorithmic Complexity [10%]
Derive and report the complexity of the chosen algorithms (in your report, not here).

### 2(b) Qualitative Evaluation [20%]
Visualize the obtained clusters using [Gephi](https://gephi.org) or any other graph visualization tool. Report the visualizations and comment your observations.

### 2(c) Quantitative Evaluation [20%]
Evaluate the quality of the clusters using label independent (topology only) metrics for both sets of graphs and label dependent metrics for graphs with labels.

*   \tipo{topology based metrics: Modularity and Conductance}
*   \tipo{label dependent metrics: NMI and ARI (report for all datasets that have labels, this should be all but **strike**)}

## 3. Evaluation using synthetic datasets: [30%]
1. Create a set of synthetic dataset using [LFR](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.community.LFR_benchmark_graph.html).  [10%]
    *   The common practice is to sample for varying values of $\mu$ which controls how well separated are the communities, i.e. generating synthetic graphs with $\mu=.1$ to $\mu=.9$, reporting average performance for 10 realizations at each difficulty level (90 total), see https://arxiv.org/abs/0805.4770, Fig 5 for example. N = 1000, or 5000 are common settings. For this experiments, you can use $\mu=.5$, n=1000, tau1 = 3, tau2 = 1.5, average degree=5, min community=20.
2. Evaluate the chosen algorithms (quantitatively using ARI/NMI) in the previous questions on this synthetic datasets and report your results [10%]
3. Compute the Average Modularity and Conductance measures for each of the following sets of datasets (i) Real-Classic, (ii) Real-Node-Label and (iii) Synthetic datasets, compare them and report your observations

## 4. [Bonus] Compare the results with an algorithm published/proposed in the last 4 years and report your observations [10%]

## 5. [Bonus] Compare the results on 3 more real world datasets not included in the assignment and report your observations [10%]